In [1]:
#imports
import numpy as np
import splat
import pandas as pd
import glob
from astropy.io import ascii
import matplotlib.pyplot as plt
from tqdm import tqdm
splat.initializeStandards()
import seaborn as sns
import popsims
splat.initializeStandards(all=True)
from scipy.interpolate import interp1d
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
%matplotlib inline


/users/caganze/research/splat


Welcome to the Spex Prism Library Analysis Toolkit (SPLAT)!
If you make use of any features of this toolkit for your research, please remember to cite the SPLAT paper:

Burgasser et al. (2017, Astro. Soc. India Conf. Series 14, p. 7); Bibcode: 2017ASInC..14....7B

If you make use of any spectra or models in this toolkit, please remember to cite the original source.
Please report any errors are feature requests to our github page, https://github.com/aburgasser/splat/




In [2]:
#load spectra, ignore binaries, objects with high uncertainty in mag and objects without parallaxes
splat_db=splat.searchLibrary(vlm=True, giant=False, young=False, binary=False)
splat_db['SHORTNAME']=splat_db.DESIGNATION.apply(lambda x: splat.designationToShortName)
sml=splat_db[~ ((splat_db.H_2MASS_E > 0.5) | (splat_db.J_2MASS_E > 0.5)) ]

#sds=sml[(sml.METALLICITY_CLASS=='sd') | (sml.METALLICITY_CLASS=='esd')  ]
sml=sml[~((sml.METALLICITY_CLASS=='sd') | (sml.METALLICITY_CLASS=='esd') \
          | (sml.MEDIAN_SNR <20))]

In [3]:
#get y dwarfs
def get_shortname(n):
        return splat.designationToShortName(n).replace('J', 'WISE')

schn='/Volumes/Lacie/schneider/*.txt'
schntb=pd.read_csv('/Volumes/Lacie/wispsdata/data/schneider2015.txt', 
                   delimiter=' ').drop(columns='Unnamed: 14')
schntb['shortname']=schntb.Name.apply(get_shortname)
spectra_schn=[]
for f in glob.glob(schn):
    d=ascii.read(f).to_pandas()
    shortname=(f.split('/')[-1]).split('.txt')[0]
    s=splat.Spectrum(wave=d.col1, 
                     flux=d.col2,
                     noise=d.col3, 
                     name=shortname)
    #measure snr 
    mask= np.logical_and(d.col1>1.0, d.col1<2.4)
    snr= (np.nanmedian(d.col2[mask]/d.col3[mask]))
    spectra_schn.append([s, snr])

In [4]:
mdwarfs=sml[ (sml.SPEX_TYPE.apply(splat.typeToNum) <20)]
ldwarfs=sml[ (sml.SPEX_TYPE.apply(splat.typeToNum).between(20, 30))]
tdwarfs=sml[ (sml.SPEX_TYPE.apply(splat.typeToNum).between(30, 40))]

In [5]:
#tighter_constraints on m dwarfs 
mdwarfs=mdwarfs[(~mdwarfs.PARALLAX.isna()) & (mdwarfs.MEDIAN_SNR >100)]
ldwarfs=ldwarfs[ (ldwarfs.MEDIAN_SNR >70)]

In [6]:
def choose_ten(df):
    if len(df) >10:
        return df.sort_values('MEDIAN_SNR', ascending=False)[:10]
    else:
        return df

In [7]:
#tdwarfs.MEDIAN_SNR

In [8]:
ls=ldwarfs.groupby('SPEX_TYPE').apply(choose_ten).reset_index(drop=True)#.groupby('SPEX_TYPE').count()

In [9]:
smlf=pd.concat([mdwarfs, ls, tdwarfs]).reset_index(drop=True)

In [10]:
def get_file(x):
    try:
        return splat.getSpectrum(filename=x)[0]
    except:
        return 

In [11]:
%%capture
templs=smlf.DATA_FILE.apply(lambda x: get_file(x))
#sds_spectra=sds.DATA_FILE.apply(lambda x:  get_file(x))


#all_spectra=pd.read_pickle('/users/caganze/research/popsims/data/splat_spectra.pkl')
#sd_spectra=pd.read_pickle('/users/caganze/research/popsims/data/sd_spectra.pkl')

In [12]:
schntb['spectra']=[x[0] for x in  spectra_schn]

schntb['snr']=[x[1] for x in  spectra_schn]

schntb=schntb[schntb.snr>=2.].reset_index(drop=True)

all_spectra=np.concatenate([templs,schntb.spectra.values ])
#all_spectra=templs

In [13]:
#import pickle
#output = open('/users/caganze/research/popsims/data/splat_spectra.pkl', 'wb')
#pickle.dump(all_spectra, output)
#output.close()

#import pickle
#output = open('/users/caganze/research/popsims/data/sd_spectra.pkl', 'wb')
#pickle.dump(sds_spectra, output)
#output.close()


In [14]:
def make_spt_number(spt):
    ##make a spt a number
    if isinstance(spt, str):
        return splat.typeToNum(spt)
    else:
        return spt

In [15]:
spts=np.concatenate([smlf.SPEX_TYPE.apply(make_spt_number).values,
                     schntb.Spec.apply(make_spt_number).values,
                    ])
#spts=smlf.SPEX_TYPE.apply(make_spt_number).values

In [16]:
assert len(spts) == len(all_spectra)

In [17]:
#ascii.write?

In [18]:
#remove nones
nones= np.array(all_spectra)==None
all_spectra=all_spectra[~nones]
spts=spts[~nones]
assert len(spts) == len(all_spectra)

In [19]:
zhang_subds=pd.read_pickle( '/volumes/LaCie/popsimsdata/combined_sd_templates.pkl')
zhang_subds['spt']=zhang_subds.sptype.apply(make_spt_number)

In [20]:
#optical_stds=pd.read_pickle('/volumes/LaCie/popsimsdata/combined_optical_templates.pkl')
#optical_stds['spt']=optical_stds.sptype.apply(make_spt_number)

In [21]:
#aurora's templates
aurora= pd.read_pickle('/volumes/LaCie/popsimsdata/popsims/aurora_templates.pkl')
aurora_dwarf_spectra = aurora.spectrum[aurora.is_dwarf].values
aurora_dwarf_spectypes= [splat.typeToNum(x) for x in aurora.proper_type[aurora.is_dwarf]]
aurora_subdwarf_spectra= aurora.spectrum[~aurora.is_dwarf].values
aurora_subdwarf_spectypes=[splat.typeToNum(x) for x in aurora.proper_type[~aurora.is_dwarf]]

In [22]:
#optical_stds

In [23]:
#only pick those ith sds or esd or usd in their sptype
zhang_subds=zhang_subds[np.logical_or.reduce([zhang_subds.sptype.str.startswith('esd'),\
                                             zhang_subds.sptype.str.startswith('sd'),\
                                              zhang_subds.sptype.str.startswith('usd')])]
#add 
#optical_sd_bool=np.logical_or.reduce([optical_stds.sptype.str.startswith('esd'),\
#                                    optical_stds.sptype.str.startswith('sd'),\
#                                    optical_stds.sptype.str.startswith('usd')]).astype(bool)


In [24]:
#zhang_subds=zhang_subds.append(optical_stds[optical_sd_bool])

In [25]:
#assert len(zhang_subds)==131

In [26]:
#zhang_subds.spt.values

In [27]:
#sd_df

In [28]:
#optical_templates=optical_stds[~optical_sd_bool].spectra.values
#optical_spts=optical_stds[~optical_sd_bool].spt.values

In [29]:
#optical_templates=optical_templates[optical_spts>=10]
#optical_spts=optical_spts[optical_spts>=10]

#assert len(np.concatenate([optical_templates, all_spectra]))==\
#len(np.concatenate([optical_spts, spts]))


In [34]:
aurora.templates

AttributeError: 'DataFrame' object has no attribute 'templates'

In [32]:
zhang_subds.spectra.values[0].name

'ULAS_J021642.96+004005.7_sdL4_Xshooter_VIS_Primeval-I.fits'

In [30]:
jk

NameError: name 'jk' is not defined

In [ ]:
#save template spectra
all_dwarf_template_dict= {'spectra':np.concatenate([all_spectra, aurora_dwarf_spectra]),
'spt':  np.concatenate([spts,  aurora_dwarf_spectypes])}

#save templates for subdwarfs 
#save template spectra
all_subdwarf_template_dict= {'spectra':np.concatenate([zhang_subds.spectra.values,\
                                                               aurora_subdwarf_spectra],
'spt': np.concatenate([zhang_subds.spt.values, aurora_subdwarf_spectypes

In [ ]:
np.save('/Volumes/LaCie/popsimsdata/dwarfs_templates.npy', [all_dwarf_template_dict], allow_pickle=True) 

In [ ]:
np.save('/Volumes/LaCie/popsimsdata/subdwarfs_templates.npy', [all_subdwarf_template_dict], allow_pickle=True) 